In [2]:
import pandas as pd

In [4]:
path_source+"88942-consumption_return.json"

'smart_heating_dataset/88942-consumption_return.json'

In [3]:
path_source = 'smart_heating_dataset/'
pd.read_json(path_source+"88942-consumption_return.json")

ValueError: If using all scalar values, you must pass an index